# Extraccion y transformacion de la data de Tecnolite API 

### Paso 1. Se descarga la lista de skus con sus estados

In [1]:
from urllib.request import Request, urlopen  
from urllib.error import URLError, HTTPError, ContentTooShortError 
import urllib.parse
from collections import Counter
import json
import copy
import csv

""" Aniadiendo un valor a la variable apiKey """
apiKey = 'povnHJRYuDufMP5i1vJnYNREAShWXgeS'

In [ ]:
""" Manda una request a la api de tecnolite """
req = Request('https://tecnolite-preprod.apimanagement.us3.hana.ondemand.com:443/pro/v1/tecnoliteb2c/products/list')
""" Le aniadimos en los headers la apikey con el valor que ya antes habiamos hecho """
req.add_header('ApiKey', apiKey)
""" 130632 """
content = urlopen(req).read()
""" Aniadiamos todo para qe lo pueda agregar en la variable y todo junto los acentos, llaves lo mantenga """
json_products = json.loads(content.decode('utf-8'))

### Se totaliza por estados. Se totaliza los productos no disponibles, es decir, los que tienen estados Y4, Y3, Z3. 

In [ ]:

products_list = json_products['ZB2C_MAESTROMATERIALES.Response']['TMAESTRO_MATERIALES']['item']
p_list = []
s_list = []
for p in products_list:
    """ Crear un objeto que tiene la propiedad code(MATNR) y la propiedad status(MSTAE) """
    product_ref = {'code' : p['MATNR'], 'status': p['MSTAE']}
    p_list.append(product_ref)
    s_list.append(p['MSTAE'])

print('Totales por estado:')
count_status = Counter(s_list)
print(count_status)
available = 0
no_available = 0
for k in count_status:
    if(k not in ['Y4', 'Y3', 'Z3']):
        available += count_status[k]
    else: 
        no_available += count_status[k]
print('Total productos: ', len(p_list))
print('Productos disponibles: ', available)
print('Productos no disponibles (con estados Y4, Y3, Z3): ', no_available)
print('\n')
print('Muestra de los 10 primeros registros:')
print('\n')
print(p_list[:10])

Totales por estado:
Counter({'': 2193, 'Y3': 1021, 'Y2': 444, 'Z2': 373, 'Z3': 238, 'Z1': 13})
Total productos:  4282
Productos disponibles:  3023
Productos no disponibles (con estados Y4, Y3, Z3):  1259


Muestra de los 10 primeros registros:


[{'code': '000000000000026169', 'status': ''}, {'code': '000000000000026171', 'status': ''}, {'code': '000000000000042578', 'status': 'Y2'}, {'code': '000000000000042580', 'status': 'Y3'}, {'code': '000000000000042636', 'status': 'Y2'}, {'code': '000000000000042637', 'status': 'Y2'}, {'code': '000000000000042751', 'status': 'Z3'}, {'code': '000000000000042752', 'status': ''}, {'code': '000000000000042753', 'status': 'Y3'}, {'code': '000000000000042755', 'status': 'Y3'}]


### Consulta del detalle de cada producto disponible

In [4]:
p_bad = []
p_detail = []
i = 0
t = len(p_list)
t_urlError = 0
t_httpError = 0

In [6]:
t_spacesError = 0
for p in p_list: 
    i += 1
    #spaces = 0
    #for a in p['code']:
        #if (a.isspace()) == True:
            #spaces += 1
    #if(spaces == 0): 
    url = 'https://tecnolite-preprod.apimanagement.us3.hana.ondemand.com:443/pro/v1/tecnoliteb2c/products/' + urllib.parse.quote_plus(p['code'].replace('/', '_')) + '?fields=FULL'
    req = Request(url)
    req.add_header('ApiKey', apiKey)
    try:
        conn = urlopen(req)
    except HTTPError as e:
        # Return code error (e.g. 404, 501, ...)
        # ...
        p_bad.append(p)
        t_httpError += 1
        print("HTTPError producto " + str(p) + ' ... ' + str(i) + ' de ' + str(t) + '.', "Error code: " + str(e.code))
    except URLError as e:
        # Not an HTTP-specific error (e.g. connection refused)
        # ...
        p_bad.append(p)
        t_urlError += 1
        print("URLError producto " + str(p) + ' ... ' + str(i) + ' de ' + str(t), e.reason)
    else:
        content = conn.read()
        JSON_detail = json.loads(content.decode('utf-8'))
        JSON_detail['status'] = p['status']
        p_detail.append(JSON_detail)
        print('Producto ' + str(i) + ' de ' + str(t))
        #print(url)
#else: 
    #p['reason'] = 'Contiene espacio en blanco.'
    #p_bad.append(p)
    #print("Error Espacios en blanco " + str(p) + ' ... ' + str(i) + ' de ' + str(t))
    #if(i==20): break

Producto 1 de 4282
Producto 2 de 4282
HTTPError producto {'code': '000000000000042578', 'status': 'Y2'} ... 3 de 4282. Error code: 400
HTTPError producto {'code': '000000000000042580', 'status': 'Y3'} ... 4 de 4282. Error code: 400
Producto 5 de 4282
HTTPError producto {'code': '000000000000042637', 'status': 'Y2'} ... 6 de 4282. Error code: 400
HTTPError producto {'code': '000000000000042751', 'status': 'Z3'} ... 7 de 4282. Error code: 400
Producto 8 de 4282
HTTPError producto {'code': '000000000000042753', 'status': 'Y3'} ... 9 de 4282. Error code: 400
HTTPError producto {'code': '000000000000042755', 'status': 'Y3'} ... 10 de 4282. Error code: 400
HTTPError producto {'code': '000000000000042756', 'status': 'Y3'} ... 11 de 4282. Error code: 400
HTTPError producto {'code': '000000000000042757', 'status': 'Y3'} ... 12 de 4282. Error code: 400
HTTPError producto {'code': '000000000000042758', 'status': 'Y3'} ... 13 de 4282. Error code: 400
HTTPError producto {'code': '000000000000042759

In [7]:
with open('tecnolite_result_v5.json', 'w', encoding='utf-8') as f:
    json.dump(p_detail, f, ensure_ascii=False, indent=4)
    
with open('tecnolite_result_bad_v5.json', 'w', encoding='utf-8') as f:
    json.dump(p_bad, f, ensure_ascii=False, indent=4)

## Resumen del proceso
print('\n')
print('ESTADISTICAS DEL PROCESO: ')
print('Productos descargados exitosamente: ', len(p_detail))
print('Productos no descargados por cualquier error: ', len(p_bad))
print('Productos no descargados por HTTPError: ', t_httpError)
print('Productos no descargados por URLError: ', t_urlError)



ESTADISTICAS DEL PROCESO: 
Productos descargados exitosamente:  2394
Productos no descargados por cualquier error:  1888
Productos no descargados por HTTPError:  1888
Productos no descargados por URLError:  0


### A modo informativo se imprimen los features que tecnolite maneja  -----

In [8]:
# Opening JSON file
f = open('tecnolite_result_v5.json')
# returns JSON object as 
# a dictionary
data = json.load(f)
sku_template = {}

for p in data: 
    if('classifications' in p):
        for f in p['classifications'][0]['features']:
            sku_template[f['name']] = ''
    if('categories' in p):
        ci = 1
        for c in p['categories']:
            sku_template['category_' + str(ci)] = ''
            ci +=1
    if('images' in p):
        im = 1
        for image in p['images']:
            if image['format'] == 'zoom':
                sku_template['image_' + str(im)] = ''
                im +=1
print(sku_template)


{'Material': '', 'Aplicación': '', 'Color de luz': '', 'Marca': '', 'Terminado': '', 'image_1': '', 'image_2': '', 'Tiempo de vida': '', 'Ángulo de dispersión': '', 'Flujo luminoso': '', 'Familia': '', 'Voltaje': '', 'Corriente': '', 'Temperatura de operación': '', 'IRC': '', 'Factor de potencia': '', 'Outlet': '', 'Nombre comercial': '', 'Inteligente': '', 'Luz Led integrada': '', 'Tipo de rosca': '', 'Potencia': '', 'IP': '', 'IK': '', 'image_3': '', 'image_4': '', 'Dimensiones del producto': '', 'Emplea Num y modelo de lámparas': '', 'Especificación eléctrica': '', 'Garantía': '', 'Atenuable / Dimerizable': '', 'Montaje': '', 'Temperatura de Color': '', 'image_5': '', 'image_6': '', 'image_7': '', 'Comparación con incandescente': '', 'Observaciones': '', 'Peso del producto': '', 'Tecnología': '', 'Tipo de bulbo': '', 'Forma del Luminario': '', 'image_8': '', 'image_9': '', 'Producto sugerido': '', 'category_1': '', 'category_2': '', 'category_3': '', 'Corte de empotramiento': '', 'i

### Prepara parseado a json segun estructura de Voltz

In [10]:
      
i = 0
data_result = []
level = 0
for p in data: 
    sku = {}
    nombre_comercial = ''
    
    sku['sku'] = p['name']
    sku['long_description'] = p['description']
    sku['sale_unit'] = 'pieza'
    sku['sale_value'] = 1
    for element in p['baseOptions']:
        if(element['selected']['code'] == p['code'] or element['selected']['code'] == p['code'].replace('/', '_')):
            sku['tech_file'] = 'https://tecnolite.mx' + element['selected']['url'] + '/dataSheet'
    sku['sku_description'] = p['summary']
    if p['status'] in ['Y3', 'Y4', 'Z3']:
        sku['status'] = 'Descontinuado'
    else:
        sku['status'] = 'Disponible'
    sku['catalog_code'] = 'C1'
    sku['source'] = 'tecnolite_api'
    sku['maker_web'] = 'https://tecnolite.mx/'
    sku['brand_favicon'] = 'https://firebasestorage.googleapis.com/v0/b/voltz-pro.appspot.com/o/public%2Fbrand_favicons%2FFAICON%20TECNOLITE.png?alt=media&token=db37c9d9-96ab-4bb7-a25f-608681a38320'
    sku['brand'] = 'Tecnolite' #Si el producto tiene otra marca, mas adelante se le pone
    sku['features'] = []
    if('classifications' in p):
        for f in p['classifications'][0]['features']:
            if f['name'] == 'Marca':
                sku['brand'] = f['featureValues'][0]['value']
            elif f['name'] == 'Garantía':
                sku['warranty'] = f['featureValues'][0]['value']
            else:
                sku['features'].append(f['name'] + ': ' + f['featureValues'][0]['value'])
    
    if sku['brand'] == 'Construlita':
        sku['catalog_code'] = 'C70'
        sku['maker_web'] = 'https://www.construlita.com/'
        sku['brand_favicon'] = 'https://firebasestorage.googleapis.com/v0/b/voltz-pro.appspot.com/o/public%2Fbrand_favicons%2FConstrulita.png?alt=media&token=98b76185-043b-4bbd-af61-b40422b1a5a1'
    sku['image_urls'] = []
    if('images' in p):
        for image in p['images']:
            if image['format'] == 'zoom' and image['imageType'] != 'PRIMARY':
                sku['image_urls'].append('https://tecnolite.mx' + image['url'])
            elif image['format'] == 'product' and image['imageType'] == 'PRIMARY':
                sku['image_cover'] = "https://tecnolite.mx" + image['url']
    
    else: 
        print('No esta: ', sku['sku'])
        
    ### Todos los productos deben tener un price_public, que es el llamado precio de lista dado por la marca, sino lo tienen
    ### luego se agrega en esta misma propiedad con el proceo de levantamiento de inventario
    
    
    sku['price_currency'] = 'MXN'
    
    if(p['price']['currencyIso'] == 'USD'):
        #tasa de cambio de 18 usd a mxn
        sku['price_public'] = round(float(p['price']['value'] *  18), 6)
    else:
        sku['price_public'] = round(float(p['price']['value']), 6)
    
    
    data_result.append(sku)
    
    i +=1
    #if(i==200) : break

print(data_result[330:350])

with open('tecnolite_to_firestore_v5.json', 'w', encoding='utf-8') as f:
    json.dump(data_result, f, ensure_ascii=False, indent=4)

No esta:  AC74240
No esta:  CO5110NBCA
No esta:  CU52951BBCA
No esta:  CU52951NBCA
No esta:  CU54951NBCA
No esta:  CU54951NBNA
No esta:  CY51951BBCA
No esta:  CY51951NBCA
No esta:  CY52951BBCA
No esta:  CY52951NBCA
No esta:  CY52951NBNA
No esta:  CY52952BBCA
No esta:  CY52952BBNA
No esta:  CY52952NBCA
No esta:  CY54951NBCA
No esta:  CY54952BBCA
No esta:  CY54952NBCA
No esta:  GTWUX18VCDNCCZ
No esta:  IN8007BBCA
No esta:  IN8007NBNA
No esta:  IN8516NBNA
No esta:  IN8517BBFA
No esta:  IN8520NBFA
No esta:  IN8526NBFA
No esta:  KITDOBATCW-ALEXA
No esta:  OU3701SBCA
No esta:  OU3701SBFA
No esta:  OU3702SBCA
No esta:  OU3702SBFA
No esta:  OU3711SBCA
No esta:  OU3711SBFA
No esta:  OU3712SBCA
No esta:  OU3712SBFA
No esta:  RE1077NBCA
No esta:  TSWI1MVBZ
No esta:  TSWI1MVNZ
No esta:  TSWI2MVBZ
No esta:  TSWI2MVNZ
No esta:  TSWI3MVBZ
No esta:  TSWI3MVNZ
No esta:  TSWI3SMVBZ
No esta:  TSWI3SMVNZ
No esta:  TSWIDIMMVBZ
No esta:  TSWIDIMMVNZ
No esta:  TZ841N000640E
No esta:  TZ842B30A330A
No esta:  